# es 1

In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import matplotlib as ptlt
import plotly.express as px
%matplotlib inline

## lettura da csv

In [118]:
df_solar_panel=pd.read_csv("solar_panels.csv",sep=";")
df_solar_panel["DaySum"]=pd.to_numeric(df_solar_panel["DaySum"])

In [119]:
df_solar_panel["Date"]=pd.to_datetime(df_solar_panel["Date"]).dt.date

In [120]:
df_kwh_giorno=pd.DataFrame(df_solar_panel.groupby("Date").max("DaySum")["DaySum"]/1000)

In [121]:
len(df_kwh_giorno)

365

In [122]:
px.bar(df_kwh_giorno,
       y="DaySum",
       template='plotly_white',
       labels={"DaySum":"Somma kW/h","index":"Tempo (giorni)"},
       color_discrete_sequence=["brown"],
       title="Produzione energetica totale giornaliera")

In [123]:
df_kwh_giorno["Date"]=pd.to_datetime(df_kwh_giorno.index)

In [124]:
df_kwh_giorno

,DaySum,Date
Date,,
2015-01-01,12.204,2015-01-01
2015-01-02,11.447,2015-01-02
2015-01-03,18.222,2015-01-03
2015-01-04,37.805,2015-01-04
2015-01-05,5.912,2015-01-05
...,...,...
2015-12-27,1.727,2015-12-27
2015-12-28,2.495,2015-12-28
2015-12-29,3.985,2015-12-29


In [125]:
df_kwh_settimana=pd.DataFrame(df_kwh_giorno["Date"].dt.isocalendar())
df_kwh_settimana["DaySum"]=df_kwh_giorno["DaySum"]
df_kwh_settimana=pd.DataFrame(df_kwh_settimana.groupby("week").sum("DaySum")["DaySum"])


In [126]:
fig = px.bar(df_kwh_settimana,
             y="DaySum",
             template='plotly_white',
             labels={"DaySum":"Somma kW/h","week":"Numero settimane"},
             color_discrete_sequence=["green"],
             title="Produzione energetica totale settimanale")
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [x for x in df_kwh_settimana.index]
    )
)
fig.show()

In [127]:
df_kwh_mese = df_kwh_giorno.groupby(df_kwh_giorno["Date"].dt.month).sum("DaySum")

In [133]:
fig = px.bar(df_kwh_mese,
             y="DaySum",
             template='plotly_white',
             labels={"DaySum":"Somma kW/h","Date":"Mesi"},
             color_discrete_sequence=["orange"],
             title="Produzione energetica totale mensile")
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [x for x in df_kwh_mese.index],
        ticktext = ["Gen","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
    )
)
fig.show()

In [159]:
df_kwh_stagioni = pd.DataFrame(index=["Primavera","Estate","Autunno","Inverno"],columns=["DaySum"]).fillna(0)
df_kwh_stagioni
inizio_primavera = pd.to_datetime("2015-03-21")
inizio_estate = pd.to_datetime("2015-06-21")
inizio_autunno = pd.to_datetime("2015-09-22")
inizio_inverno = pd.to_datetime("2015-12-22")



In [160]:
kwh_totale = sum(df_kwh_giorno.DaySum)

In [161]:
df_kwh_stagioni.loc["Primavera","DaySum"] = sum(df_kwh_giorno[(df_kwh_giorno.Date>=inizio_primavera) & (df_kwh_giorno.Date<inizio_estate)].DaySum)
df_kwh_stagioni.loc["Estate","DaySum"] = sum(df_kwh_giorno[(df_kwh_giorno.Date>=inizio_estate) & (df_kwh_giorno.Date<inizio_autunno)].DaySum)
df_kwh_stagioni.loc["Autunno","DaySum"] = sum(df_kwh_giorno[(df_kwh_giorno.Date>=inizio_autunno) & (df_kwh_giorno.Date<inizio_inverno)].DaySum)
df_kwh_stagioni.loc["Inverno","DaySum"] = kwh_totale - sum(df_kwh_stagioni.DaySum)

In [169]:
fig = px.bar(df_kwh_stagioni,
             y="DaySum",
             template='plotly_white',
             labels={"DaySum":"Somma kW/h","index":"Stagioni"},
             color_discrete_sequence=["orange"],
             title="Produzione energetica totale stagionale",
             width=700,
             height=400,)
for data in fig.data:
    data["width"] =0.5
fig.show()

In [191]:
fig = px.bar(df_kwh_stagioni.transpose(),
             template='plotly_white',
             labels={"value":"Somma kW/h","index":"", "variable":"Stagioni"},
             color_discrete_sequence= px.colors.qualitative.Plotly,
             title="Produzione energetica totale",
             width=700,
             height=400,
             barmode = "stack")
fig.update_xaxes(showticklabels=False)
for data in fig.data:
    data["width"] =0.3
fig.show()

In [203]:
px.pie(df_kwh_stagioni,  
        values='DaySum', 
        names=df_kwh_stagioni.index,
        title=f"Influenza delle stagioni sul totale ({int(kwh_totale)}kwh)",
        width=700,
        height=400,)